In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torchvision
import shutil 

batch_size= 32

mean = -0.1
std=1
train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('../datasets/mnist', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                                torchvision.transforms.Resize([32, 32]),
                                torchvision.transforms.ToTensor(),
                                torchvision.transforms.Normalize(
                                 (mean,), (std,))
                             ])),
  batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('../datasets/mnist', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                                torchvision.transforms.Resize([32, 32]),
                                torchvision.transforms.ToTensor(),
                                torchvision.transforms.Normalize(
                                 (mean,), (std,))
                             ])),
  batch_size=16, shuffle=False, drop_last= True)

for x,y in train_loader:
    break
    
vmin= x.min().item()
vmax= x.max().item()
print('range : ',vmin, vmax)

range :  0.10000000149011612 1.100000023841858


In [2]:
import torch
import shutil 
import numpy as np
from torch import nn
from modules.kernels import get_gaussian
from modules.models.decoder import simple_generator
from modules.models.forward_model import forward_modelA
from modules.models.forward_H import modelH
from modules.train_utils import train
from modules.custom_activations import inc_m
import matplotlib.pyplot as plt
from modules.models.preprocess_H_weights import ifft_2d_with_fftshift_real
from modules.noise import poisson_noise
from modules.models.classifiers import simple_mnist_classifier

device='cuda' if torch.cuda.is_available() else 'cpu'

classifier = simple_mnist_classifier(32).to(device)
classifier.load_state_dict(torch.load('saved_models/mnist_classifier.pth', map_location=device)['model_state_dict'])

<All keys matched successfully>

In [3]:
img_size= 32
m_inc_epoc= 1

def inc_1_after_100_interval_10(m, epoch):
    if epoch>150 and epoch%10==0:
        m=inc_m(m, epoch, 1)
    return m

sPSF= torch.tensor(get_gaussian(side_len=5, s=1)).float().to(device)
exPSF= torch.tensor(get_gaussian(side_len=5, s=1)).float().to(device)

criterion= nn.L1Loss().to(device)
train_model_iter, train_H_iter= 1, 1
epochs=200
show_results_epoch=1
initialization_bias= 10

## Test run

In [ ]:
'''
from modules.custom_activations import sigmoid_custom as H_activation

T=5
H_init= 'fft'
H_weight_preprocess= ifft_2d_with_fftshift_real
m_inc_proc =inc_1_after_100_interval_10

decoder= simple_generator(T, img_size).to(device)
opt_model= torch.optim.Adam(decoder.parameters(), lr= 0.001)

H_generator = modelH(T, img_size = img_size, preprocess_H_weights= H_weight_preprocess, device = device, initialization_bias=0, activation = H_activation, init_method= H_init).to(device)
opt_H= torch.optim.Adam(H_generator.parameters(), lr= 0.01)

noise=True # gradient exploding is not there (because I have added change the image range [1,2] -> which not contains 0s in it: forward_model.py)
train_model_iter, train_H_iter= 1, 1

train(decoder, forward_modelA, H_generator, sPSF, exPSF, criterion, [opt_model, opt_H], train_loader, test_loader, device, T, 10, 1, train_model_iter, train_H_iter, m_inc_epoc, m_inc_proc, './1', noise, classifier, [mean, std])

'''

## Experiments

In [9]:
!rm -rf 'figs/mnistv3'
!mkdir 'figs/mnistv3'

In [ ]:
from modules.custom_activations import sigmoid_custom

save_path= 'figs/mnistv3'

H_init = 'fft'
H_weight_preprocess= ifft_2d_with_fftshift_real
m_inc_proc =  inc_1_after_100_interval_10
initialization_bias= 0
H_activation= sigmoid_custom
noise_K=1000

Ts=[5, 2, 16]
noises= [True, False]
lrs_H= [0.01, 0.5, 0.005]

for T in Ts:
    for noise in noises:
        for lr_H in lrs_H:
            try:
                if H_weight_preprocess ==None:H_weight_preprocess_name = 'None'
                else:H_weight_preprocess_name = H_weight_preprocess.__name__
                if m_inc_proc==None:m_inc_proc_name = 'None'
                else:m_inc_proc_name = m_inc_proc.__name__

                save_dir= f'{save_path}/T({T})@noise({noise})@lr_H({lr_H})'
                print(f'\n\nRUNNING EXPERIMENT :: {save_dir}')

                try:shutil.rmtree(save_dir)
                except:pass

                decoder= simple_generator(T, img_size).to(device)
                opt_model= torch.optim.Adam(decoder.parameters(), lr= 0.001)

                H_generator = modelH(T, img_size = img_size, preprocess_H_weights= H_weight_preprocess, device = device, initialization_bias=initialization_bias, activation = H_activation, init_method= H_init).to(device)
                opt_H= torch.optim.Adam(H_generator.parameters(), lr= lr_H)

                train(decoder, forward_modelA, H_generator, sPSF, exPSF, criterion, [opt_model, opt_H], train_loader, test_loader, device, T, epochs, show_results_epoch, train_model_iter, train_H_iter, m_inc_epoc, m_inc_proc, save_dir, noise, classifier, [mean, std], noise_K)

            except Exception as e:
                print(f'ERROR :: {save_dir} :: {e}')
                pass